In [10]:
# VIP Customer Classification with Reverse ETL (Sales + Sentiment Data)
# ---------------------------------------------------------------------
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import sqlalchemy
import json

# ---------------- Step 1: Load CSV & JSON ----------------
sales_df = pd.read_csv("./raw_data/sale_price.csv")

with open("./raw_data/customer_feedback.json", "r") as f:
    sentiments_data = json.load(f)

sentiments_df = pd.DataFrame(sentiments_data)
# ---------------- Step 2: Feature Engineering from Sales ----------------
# Convert sale_price to numeric (remove $ sign)
sales_df['sale_price'] = sales_df['sale_price'].replace('[\$,]', '', regex=True).astype(float)

# Compute per-customer metrics
agg_sales = sales_df.groupby('customer_id').agg(
    total_purchase_amount=('sale_price', 'sum'),
    purchase_count=('sale_id', 'count'),
    average_transaction_value=('sale_price', 'mean')
).reset_index()

# Purchase frequency: purchases per unique month in dataset
sales_df['sale_month'] = pd.to_datetime(sales_df['sale_date'], errors='coerce').dt.to_period('M')
freq = sales_df.groupby('customer_id')['sale_month'].nunique().reset_index(name='purchase_frequency')

# Merge the frequency into aggregated sales
agg_sales = agg_sales.merge(freq, on='customer_id', how='left')

# ---------------- Step 3: Merge with Sentiments ----------------
# If multiple reviews per customer, average sentiment
agg_sentiments = sentiments_df.groupby('customer_id', as_index=False).agg(
    sentiment_score=('sentiment_score', 'mean')
)

combined_df = pd.merge(agg_sales, agg_sentiments, on="customer_id", how="left").fillna(0)

print("Combined Customer-Level Data:")
print(combined_df.head())

# ---------------- Step 4: Preprocess ----------------
features = ['total_purchase_amount', 'purchase_frequency', 'average_transaction_value', 'sentiment_score']

scaler = MinMaxScaler()
combined_df[features] = scaler.fit_transform(combined_df[features])

# ---------------- Step 5: K-Means Clustering ----------------
X = combined_df[features]
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
combined_df['vip_cluster'] = kmeans.fit_predict(X)

# Identify VIP cluster based on highest spending
vip_cluster_label = combined_df.groupby('vip_cluster')['total_purchase_amount'].mean().idxmax()
combined_df['VIP_Status'] = combined_df['vip_cluster'].apply(lambda x: 'VIP' if x == vip_cluster_label else 'Non-VIP')

# ---------------- Step 6: Reverse ETL (Export) ----------------
combined_df.to_csv("data_warehouse/enriched_customer_data.csv", index=False)
print("\n[INFO] Enriched data saved to 'enriched_customer_data.csv'.")

C:\Users\skrut\AppData\Local\Temp\ipykernel_15700\1278051323.py:28: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sales_df['sale_month'] = pd.to_datetime(sales_df['sale_date'], errors='coerce').dt.to_period('M')
C:\Users\skrut\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Combined Customer-Level Data:
  customer_id  total_purchase_amount  purchase_count  \
0       C_001                  520.5               3   
1       C_002                   75.0               1   
2       C_003                  270.0               2   
3       C_004                   30.0               1   
4       C_005                   75.0               1   

   average_transaction_value  purchase_frequency  sentiment_score  
0                      173.5                   2         3.600000  
1                       75.0                   0         4.316667  
2                      135.0                   1         2.900000  
3                       30.0                   1         2.928571  
4                       75.0                   1         3.728571  

[INFO] Enriched data saved to 'enriched_customer_data.csv'.
